In [ ]:
import pandas as pd

charge_capture_df=pd.read_excel("D:\payroll\Charge Capture - Jul 20th - Aug 2nd.xlsx")
sorted_fac = sorted(charge_capture_df['Facility'].unique())

import re
metadata = {}
pattern = re.compile(r'\b9930\d{2}\b')  # Example pattern for 6-digit codes starting with 9930
cpt_time_dict = {
    "99304": 60,
    "99305": 75,
    "99306": 90,
    "99307": 15,
    "99308": 25,
    "99309": 35,
    "99310": 45
}
Facilites_groups=charge_capture_df.groupby('Facility', as_index=False)

<>:3: SyntaxWarning: invalid escape sequence '\C'
<>:3: SyntaxWarning: invalid escape sequence '\C'
C:\Users\Lenovo\AppData\Local\Temp\ipykernel_9128\1203768819.py:3: SyntaxWarning: invalid escape sequence '\C'
  charge_capture_df=pd.read_excel("D:\Charge Capture - Jul 1st - Jul 30th.xlsx")


In [3]:
from collections import defaultdict
stopwords = {"of", "the", "at", "and", "care", "center", "rehab", "nursing", "health", "llc", "snf", "m", "by", "for", "on", "in", "living", "villa", "home"}

# Example: group by the first word in the facility name
grouped_facilities = defaultdict(list)
for fac in sorted_fac:
    key = fac.split()[0]  # or use a different logic for matching
    if key.lower() in stopwords:
        key = fac.split()[1]
    grouped_facilities[key].append(fac)
corporate_groups = {k: v for k, v in grouped_facilities.items() if len(v) > 1}
single_facilities = {k: v for k, v in grouped_facilities.items() if len(v) == 1}
keys_to_remove = []
# Try to merge singles into corporate groups if their key matches
for key, facilities in single_facilities.items():
    for corp_key in corporate_groups:
        if corp_key.lower() in single_facilities[key][0].lower():
            corporate_groups[corp_key].extend(facilities)
            keys_to_remove.append(key)
            break
for key in keys_to_remove:
    if key in single_facilities:
        del single_facilities[key]
# # Print results
# for key, facilities in corporate_groups.items():
#     print(f"Corporate group '{key}': {facilities}")

# for key, facilities in single_facilities.items():
#     print(f"Single facility '{key}': {facilities}")


In [29]:
for key, facilities in corporate_groups.items():
    metadata[key] = {}  # Initialize the dictionary for this key
    metadata[key]['Facility_Data'] = {}
    corp_data = pd.DataFrame()
    facility_summary = []
    for facility in facilities:
        print(f"Processing Corporate Group: {key}, Facility: {facility}")
        group = Facilites_groups.get_group(facility).copy()
        group['CPT Codes'] = group['CPT Codes'].astype(str).str.split(',')
        group = group.explode('CPT Codes')
        group['CPT Codes'] = group['CPT Codes'].str.strip()
        Facility_Level = group.groupby(['Provider', 'CPT Codes', 'DOS']).size().reset_index(name='Count')
        Facility_Level['Minutes'] = Facility_Level['CPT Codes'].astype(str).map(cpt_time_dict) * Facility_Level['Count']
        # Remove rows where Minutes == 0 or is NaN
        Facility_Level = Facility_Level[(Facility_Level['Minutes'] != 0) & (Facility_Level['Minutes'].notna())]
        Facility_Level_result = Facility_Level.groupby(['Provider', 'DOS'], as_index=False)['Minutes'].sum()
        Facility_Level_result['Hours'] = Facility_Level_result['Minutes'] / 60
        # Group by Provider for this facility and sum hours
        provider_hours = Facility_Level_result.groupby('Provider', as_index=False)['Hours'].sum()
        provider_hours['Facility'] = facility
        facility_summary.append(provider_hours)
        metadata[key]['Facility_Data'][facility] = Facility_Level_result
    # Concatenate all facility summaries
    summary_df = pd.concat(facility_summary, ignore_index=True)
    # Group by Facility, aggregate clinicians and sum hours
    summary_grouped = summary_df.groupby('Facility').agg({
        'Provider': lambda x: '\n'.join(x),
        'Hours': 'sum'
    }).reset_index()
    summary_grouped = summary_grouped.rename(columns={'Provider': 'Clinician'})
    metadata[key]['Corp_data'] = summary_grouped
    print(summary_grouped)
    break

Processing Corporate Group: Accolade, Facility: Accolade Healthcare of Danville
Processing Corporate Group: Accolade, Facility: Accolade Healthcare of Paxton Senior Living - SNF
Processing Corporate Group: Accolade, Facility: Accolade Healthcare of Paxton on Pells
Processing Corporate Group: Accolade, Facility: Accolade Healthcare of Peoria (M)
Processing Corporate Group: Accolade, Facility: Accolade Healthcare of Pontiac LLC
                                            Facility  \
0                    Accolade Healthcare of Danville   
1  Accolade Healthcare of Paxton Senior Living - SNF   
2             Accolade Healthcare of Paxton on Pells   
3                  Accolade Healthcare of Peoria (M)   
4                 Accolade Healthcare of Pontiac LLC   

                      Clinician      Hours  
0             Stephen Skimehorn  50.416667  
1                 Christina Hay  73.166667  
2                 Christina Hay  64.916667  
3                     Eli Perez  18.083333  
4  Chris

In [30]:
group = Facilites_groups.get_group( 'Accolade Healthcare of Pontiac LLC')
provider=group[['Provider', 'CPT Codes', 'DOS']]
provider['CPT Codes'] = provider['CPT Codes'].astype(str).str.split(',')
provider_exploded = provider.explode('CPT Codes')
provider_exploded['CPT Codes'] = provider_exploded['CPT Codes'].str.strip()
provider_exploded
counts = group[['Provider', 'CPT Codes']].value_counts().reset_index(name='Count')
counts

C:\Users\Lenovo\AppData\Local\Temp\ipykernel_9128\3773572535.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  provider['CPT Codes'] = provider['CPT Codes'].astype(str).str.split(',')


,Provider,CPT Codes,Count
0,Paige Dennis,99308,65
1,Paige Dennis,99309,34
2,Christina Hay,99308,8
3,Paige Dennis,99306,6
4,Christina Hay,99305,3
5,Paige Dennis,"99306,44444",3
6,Hina Patel,99495,1
7,Paige Dennis,99305,1
8,Roshana Brown,99495,1


In [31]:
filtered = provider_exploded[(provider_exploded['Provider'] == 'Hina Patel')]
cpt_time_dict = {
    "99304": 60,
    "99305": 75,
    "99306": 90,
    "99307": 15,
    "99308": 25,
    "99309": 35,
    "99310": 45
}
sum=150+(13*35)

In [32]:
filtered

,Provider,CPT Codes,DOS
16299,Hina Patel,99495,07/03/2025


In [33]:
metadata["Accolade"]

{'Facility_Data': {'Accolade Healthcare of Danville':             Provider         DOS  Minutes      Hours
  0  Stephen Skimehorn  07/02/2025      175   2.916667
  1  Stephen Skimehorn  07/07/2025      650  10.833333
  2  Stephen Skimehorn  07/09/2025      250   4.166667
  3  Stephen Skimehorn  07/14/2025      675  11.250000
  4  Stephen Skimehorn  07/16/2025      100   1.666667
  5  Stephen Skimehorn  07/21/2025      775  12.916667
  6  Stephen Skimehorn  07/23/2025      400   6.666667,
  'Accolade Healthcare of Paxton Senior Living - SNF':         Provider         DOS  Minutes      Hours
  0  Christina Hay  07/01/2025      510   8.500000
  1  Christina Hay  07/03/2025      425   7.083333
  2  Christina Hay  07/10/2025      870  14.500000
  3  Christina Hay  07/15/2025      750  12.500000
  4  Christina Hay  07/17/2025      555   9.250000
  5  Christina Hay  07/22/2025      610  10.166667
  6  Christina Hay  07/24/2025      670  11.166667,
  'Accolade Healthcare of Paxton on Pells':  